In [ ]:
%load_ext sql
%sql sqlite:///tmcp.db

In [ ]:
%%sql

SELECT * FROM sqlite_master WHERE "type"='table';

In [1]:
import sqlite3
import re
import copy

In [2]:
def player_name_initials(_player):
    ini_exp = '^([A-Z]).+\s([A-Z])'
    ini_grp = re.findall(ini_exp, _player)
    initials = ''
    for i in ini_grp[0]:
        initials = initials+i
    criteria = (_player, _player, initials)
    return criteria

In [3]:
def get_query(_query, query_list):
    conn = sqlite3.connect(sqldb_path)
    c = conn.cursor()
    c.execute(_query, query_list)

    _subq = c.fetchall()
    conn.close()
    return _subq

In [4]:
def serves_by_location(_fs, _ss):
    # All first and second serves compiled by location
    fs_t = {'4': 0, '5': 0, '6': 0}
    for i, j in _fs.items():
        for k in j:
            fs_t[k] += j[k]

    ss_t = {'4': 0, '5': 0, '6': 0}
    for i, j in _ss.items():
        for k in j:
            ss_t[k] += j[k]

    total_fs = sum(fs_t.values())
    total_ss = sum(ss_t.values())

    fs_p, ss_p, as_p = {}, {}, {}  # first serve percentages by position

    for i in fs_t:
        fs_p[i] = fs_t[i]/total_fs
    for i in ss_t:
        ss_p[i] = ss_t[i]/total_ss

    for i, j in zip(fs_t, ss_t):
        as_p[i] = (fs_t[i]+ss_t[i]) / (total_ss + total_fs)

    fs_p = rename_num_to_pos(fs_p)
    ss_p = rename_num_to_pos(ss_p)
    as_p = rename_num_to_pos(as_p)

    return fs_p, ss_p, as_p

In [5]:
def rename_num_to_pos(di):
    di['Wide'] = di.pop('4')
    di['Body'] = di.pop('5')
    di['Down T'] = di.pop('6')
    return di

In [6]:
def single_break_points(_fs, _ss):
    bpfs = {'30-40':{}, '40-AD':{}}
    bpss = {'30-40': {}, '40-AD': {}}
    for i in _fs:
        if i in bpfs:
            bpfs[i] = _fs[i]
    for i in _ss:
        if i in bpss:
            bpss[i] = _ss[i]

    _sbp_srv1p, _sbp_srv2p, _sbp_srvp = serves_by_location(bpfs,bpss)

    return _sbp_srv1p, _sbp_srv2p, _sbp_srvp

In [7]:
def multi_break_points(_fs, _ss):
    bpfs = {'0-40': {}, '15-40': {}}
    bpss = {'0-40': {}, '15-40': {}}
    for i in _fs:
        if i in bpfs:
            bpfs[i] = _fs[i]
    for i in _ss:
        if i in bpss:
            bpss[i] = _ss[i]

    _mbp_srv1p, _mbp_srv2p, _mbp_srvp = serves_by_location(bpfs,bpss)

    return _mbp_srv1p, _mbp_srv2p, _mbp_srvp

In [8]:
def ret_serves(_player):
    ql = player_name_initials(_player)  # query list
    qry = 'SELECT serving, pts, first, second FROM all_points WHERE (player1 = ? OR player2 = ?) AND serving = ?;'
    subq = get_query(qry, ql)

    fs = AllServes(all_points)
    ss = AllServes(all_points)

    # all first and second serves compiled by score
    # the extra conditions are due to dirty data
    # sometimes the scores are wrong (e.g. 2-1 instead of 30-40)
    # or the directions are wrong (not 4,5,6)
    for i in subq:
        if i[2] and (i[1] in fs.pts) and (i[2][0] in fs.pts[i[1]]):
            fs.pts[i[1]][i[2][0]] += 1
        if i[3] and (i[1] in ss.pts) and (i[3][0] in ss.pts[i[1]]):
            ss.pts[i[1]][i[3][0]] += 1
        # else:
        #   add a dictionary to catch all the skipped rows

    # for i, j in zip(fs.pts, ss.pts):
    #     print(i, fs.pts[i], ss.pts[j])

    all_srv1p, all_srv2p, all_srvp = serves_by_location(fs.pts, ss.pts)
    sbp_srv1p, sbp_srv2p, sbp_srvp = single_break_points(fs.pts, ss.pts)
    mbp_srv1p, mbp_srv2p, mbp_srvp = multi_break_points(fs.pts, ss.pts)

    _compiled_serves = {'all_serves': {'srv1': all_srv1p, 'srv2': all_srv2p, 'all': all_srvp},
                       'sbp_serves': {'srv1': sbp_srv1p, 'srv2': sbp_srv2p, 'all': sbp_srvp},
                       'mbp_serves': {'srv1': mbp_srv1p, 'srv2': mbp_srv2p, 'all': mbp_srvp}}

    return _compiled_serves

In [ ]:
b = AllServes()

for i in b.pts:
    print(i, b.pts[i])

In [ ]:
a = AllServes()

for i in a.pts:
    print(i, a.pts[i])

In [9]:
all_points = ('0-0', '0-15', '0-30', '0-40', '15-0', '15-15', '15-30', '15-40', '30-0',
             '30-15','30-30', '30-40', '40-0', '40-15', '40-30', '40-40', 'AD-40', '40-AD')
all_bp = ('0-40', '15-40', '30-40','40-AD')
single_bp = ('30-40','40-AD')
multi_bp = ('0-40', '15-40')

class AllServes:
    def __init__(self, scaffold):
        self.scores = {'4': 0, '5': 0, '6': 0}
        self.scaffold = scaffold
        self.pts = {i:copy.deepcopy(self.scores) for i in self.scaffold}

class RetDir:
    def __init__(self):
        self.ret = {1:0, 2:0, 3:0}

class ServDir:
    def __init__(self):
        self.dir = [4, 5, 6]
        self.srv = {i:RetDir() for i in self.dir}

class AllReturns:
    def __init__(self, scaffold):
        self.scaffold = scaffold
        self.pts = {i:ServDir() for i in self.scaffold}
    
    def __str__(self):
        bp = ''
        for i in self.pts:
            bp +='{0:5s}: '.format(i)
            for j in self.pts[i].srv:
                bp += '  {}- '.format(j)
                for k in self.pts[i].srv[j].ret:
                    bp += '{0}:{1:<4} '.format(k, self.pts[i].srv[j].ret[k])
            bp += '\n'
        return bp

In [ ]:
del b
b = AllReturns(multi_bp)

In [ ]:
print(b)

## Analyzing Serve Directions 

In [11]:
sqldb_path = 'tmcp.db'
player = 'Rafael Nadal'
compiled_serves = ret_serves(player)

as1w = round(100 * compiled_serves['all_serves']['srv1']['Wide'])
as1b = round(100 * compiled_serves['all_serves']['srv1']['Body'])
as1t = round(100 * compiled_serves['all_serves']['srv1']['Down T'])

ss1w = round(100 * compiled_serves['sbp_serves']['srv1']['Wide'])
ss1b = round(100 * compiled_serves['sbp_serves']['srv1']['Body'])
ss1t = round(100 * compiled_serves['sbp_serves']['srv1']['Down T'])

ms1w = round(100 * compiled_serves['mbp_serves']['srv1']['Wide'])
ms1b = round(100 * compiled_serves['mbp_serves']['srv1']['Body'])
ms1t = round(100 * compiled_serves['mbp_serves']['srv1']['Down T'])

as2w = round(100 * compiled_serves['all_serves']['srv2']['Wide'])
as2b = round(100 * compiled_serves['all_serves']['srv2']['Body'])
as2t = round(100 * compiled_serves['all_serves']['srv2']['Down T'])

ss2w = round(100 * compiled_serves['sbp_serves']['srv2']['Wide'])
ss2b = round(100 * compiled_serves['sbp_serves']['srv2']['Body'])
ss2t = round(100 * compiled_serves['sbp_serves']['srv2']['Down T'])

ms2w = round(100 * compiled_serves['mbp_serves']['srv2']['Wide'])
ms2b = round(100 * compiled_serves['mbp_serves']['srv2']['Body'])
ms2t = round(100 * compiled_serves['mbp_serves']['srv2']['Down T'])

asaw = round(100 * compiled_serves['all_serves']['all']['Wide'])
asab = round(100 * compiled_serves['all_serves']['all']['Body'])
asat = round(100 * compiled_serves['all_serves']['all']['Down T'])

print('1st serve, no break:', 'Wide', as1w, ', Body', as1b, ', DownT', as1t)
print('All Serves:', 'Wide', asaw, ', Body', asab, ', DownT', asat)


1st serve, no break: Wide 40 , Body 21 , DownT 39
All Serves: Wide 37 , Body 27 , DownT 36


In [ ]:
ql = player_name_initials(player)  # query list
qry = 'SELECT serving, pts, first, second FROM all_points WHERE (player1 = ? OR player2 = ?) AND serving = ?;'
subq = get_query(qry, ql)

print(subq[0:10])
print(len(subq))

## Analyzing Returns ##

In [ ]:
def filtering_returns(_player):
    
    ql = player_name_initials(_player)
    ret_qry = 'SELECT serving, pts, first, second FROM all_points WHERE (player1 = ? OR player2 = ?) AND NOT serving = ?;'
    subq = get_query(ret_qry, ql)
    # print(subq[0:10]) # for checking during testing
    
    # most shots have position (1,2,3) before depth (7,8,9), 
    # but some have depth before position so have to select it as a second number
    str ='^\d([bfsrv])([123789][123]*)'

    rl = [] # return list
    for i in subq:
        j = re.findall(str, i[2])
        if j:
            rl.append((i[1], 1, int(i[2][0]), j[0][0], int(j[0][1]), i[2]))
            # tuple is (point, serve, serve-dir, return-shot, return-dir, full-rally)
        if i[3]:
            k = re.findall(str, i[3])
            if k: # this drops any double faults as null strings are skipped
                rl.append((i[1], 2, int(i[3][0]), k[0][0],int(k[0][1]), i[3]))
    return rl

In [ ]:
player = 'Rafael Nadal'
tl = filtering_returns(player)

for i in tl[0:10]:
    print(i)

In [ ]:
def segmenting_returns(rl, points, aggregation=0):
    # rl is the returns list from filtering_returns()
    # aggregation is if you want to change the AllReturns matrix to something 
    #      different than the points you're selecting
    
    if aggregation != 0:
        bucket = aggregation
    else:
        bucket = points
    serve_returns = AllReturns(bucket)
    
    for i in rl:
        # the mod 10 is there due to dirty data. a number could be 1,2,3 or 7X,8X,9X
        # this aggregation excludes tiebreaker points (~2-3% of points)
        if (i[0] in points) and (i[2] in [4, 5, 6]) and ((i[4] % 10) in [1,2,3]):
            if aggregation == 0:
                serve_returns.pts[i[0]].srv[i[2]].ret[i[4] % 10] += 1
            else:
                # adding extra complexity to function. I don't like it
                # sums the original points bin into one aggregate bucket
                serve_returns.pts[aggregation[0]].srv[i[2]].ret[i[4] % 10] += 1
    return serve_returns

In [ ]:
a = segmenting_returns(tl, all_points)
print(a)

In [ ]:
t = 0
for i in a.pts:
    t += a.pts[i].srv[4].ret[1]
print(t)

In [ ]:
b = segmenting_returns(tl, all_bp)
print(b)

c = segmenting_returns(tl, single_bp)
print(c)

d = segmenting_returns(tl, multi_bp)
print(d)

e = segmenting_returns(tl, all_points, ['sbp'])
print(e)

for j in e.pts['sbp'].srv[i].ret:
    a = 0
    for i in e.pts['sbp'].srv:
        a += e.pts['sbp'].srv[i].ret[j]
    print(a, end=' ')
        